## 本程序统计所有结构的等效位点情况。然后将各非等效位点提取出来，组成新的数据集，新的数据集类似于“_untar”文件夹，可做统计、绘图等后续工作。本程序服务对象是untar文件夹

### 本程序只对晶体机构有意义，非晶结构尽管可以计算，但使用意义不大，因为都是不等效位点。

### 检测等效位点的方法是检查coor.dat的配位数和配位键长在给定限制下是否一致。

### 假设需要统计的文件的目录结构为“cu_crystal_different_xyz_WT_untar”（一级目录）——“Cu-bulk”（二级目录）——“1/2/3/4等解压缩的文件夹”，将本程序安放在与一级目录平行的另一个一级目录（或一级目录本身）的任意一个二级文件目录内，即可运行。

### 它会读取用户输入的一级目录（如“cu_crystal_different_xyz_WT_untar”）下所有的“coor.dat”文件，运行后会给出所有可能等效的位点集合，并在本程序所在的目录内产生一个“symmetry.dat”的文件，文件会给出所有可能等效的位点。
### 程序会在生成一级目录“cu_crystal_different_xyz_WT_untar_symmetry”，里面为提取出的所有非等效位点的文件（即从“_untar”中copy而来）。另外还在该文件夹下有一个“symmetry.dat”的文件，给出“_untar”中各体系的等效位点统计情况。

如果是Cu体系，考虑到Cu最近邻距离是2.553A，EXAFS默认计算为其2.2倍内的原子散射情况，即5.62A，Cu面心立方结构的第四配位距离为5.105A，第5配位为5.708A，第6配位为6.253A，故最多考虑到第5配位。

last update： 2021.9.15

contacts: zhaohf@ihep.ac.cn

# 配置环境

## 导入各种模块

In [1]:
import os
import sys
import pandas as pd
import shutil

输出导入模板的版本。

In [2]:
print('python version:',sys.version)
print('panda version:',pd.__version__)
#print('shutil version:',shutil.__version__)

python version: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
panda version: 1.3.4


# 设置输入文件夹和输出文件夹

## 设置源文件夹，即要处理的文件夹

In [3]:
# 设置源文件夹，即经过计算集群计算的文件夹
dir_src = 'cu_crystal_different_xyz_WT_untar'

## 设置输出文件夹名

In [4]:
dir_work = dir_src + '_symmetry'

## 设置对称性统计文件名

In [5]:
file_out = 'symmetry.dat'

## 统计coor.dat中前shell层的数据。

In [6]:
shell = 4

# 一些函数、类

## 读取coor.dat文件中的数组

In [7]:
def read_coor(path_coor, shell=1):
    """
    读取coor.dat中前shell个壳层的数据
    """
    exist = os.path.exists(path_coor)
    if exist:
        coor = pd.read_csv(path_coor,delim_whitespace=True,header=None, skiprows=[0,1])
        coor_data = coor.values
        cn = coor_data[0:shell,1]
        r = coor_data[0:shell,2]
        r_d = coor_data[0:shell,3]
#
        return cn,r,r_d
#
    else:
        return None  

# 程序正文

## 设置工作文件夹

In [8]:
# 当前目录
path = os.path.abspath('./')
path

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup\\jupyterbook'

In [9]:
# 上一级目录
path_par = os.path.abspath(os.path.pardir)
path_par

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup'

In [10]:
# 设置源文件夹，即经过计算集群计算的文件夹
#dir_src = 'AuPd-4000_au150-0_DW_R0_cncr_WT_untar'
path_src = os.path.join(path_par, dir_src)
# 检查源文件夹是否存在，存在则列出其内部文件夹，不存在则打印错误
exist = os.path.exists(path_src)
if exist:
    list_seed = os.listdir(path_src)
else:
    print()
    print('** Error!! WT untared directory', dir_src, 'is not found!' )
    print()
#    sys.exit()
list_seed

['Cu-bulk',
 'Cu-fcc-22',
 'Cu-fcc-22-1',
 'Cu-fcc-23',
 'Cu-fcc-23-1',
 'Cu-fcc-33',
 'Cu-fcc-33-1',
 'Cu-fcc-33-2',
 'Cu-fcc-331',
 'Cu-fcc-333',
 'Cu-fcc-34-2',
 'Cu-fcc-342',
 'Cu-fcc-41',
 'Cu-fcc-42',
 'Cu-fcc-43-1',
 'Cu-fcc-43-2',
 'Cu-fcc-44-2',
 'Cu-fcc-53',
 'Cu-wulff-size13',
 'Cu-wulff-size201']

In [11]:
# 构建目标文件夹和目录
#dir_work = dir_src + '_symmetry'
path_work = os.path.join(path_par, dir_work)
os.makedirs(path_work, exist_ok=True)
path_work

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup\\cu_crystal_different_xyz_WT_untar_symmetry'

In [12]:
# 构建输出文件目录
#dir_work = dir_src + '_symmetry'
path_file_work = os.path.join(path_work, file_out)
path_file_work

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup\\cu_crystal_different_xyz_WT_untar_symmetry\\symmetry.dat'

## 统计coor.dat中前shell层的数据。

In [13]:
coor_data = []       # 列表，保存配位信息
cluster_index = []   # 列表，保留coor_data所对应配位信息的cluster的index
site_index = []      # 列表，保留coor_data所对应配位信息的cluster的site的index
#
# 进入各个cluster，即100、200等
for dir_seed in list_seed:
#
# 源seed的路径
    path_seed_src = os.path.join(path_src, dir_seed)
# 源seed中各个xyz文件夹  
    list_xyz = os.listdir(path_seed_src)
#
    for dir_xyz in list_xyz:
#
# 源xyz的路径
        path_xyz_src = os.path.join(path_seed_src, dir_xyz)
#
# 查找xyz路径（文件夹）下是否存在“coor.dat”文件
        file = 'coor.dat'
        path_coor_src = os.path.join(path_xyz_src, file)
        exist = os.path.exists(path_coor_src)
        if exist:
            coor = pd.read_csv(path_coor_src,delim_whitespace=True,header=None, skiprows=[0,1])
            coor_data.append(coor.values[0:shell])
            cluster_index.append(dir_seed)
            site_index.append(dir_xyz)
#
N_site = len(coor_data)
have_grouped = []
group = []
for i in range(N_site):
    have_grouped.append(False)
    group.append([])
#
# k is the index of group
k = -1
for i in range(N_site):
    if have_grouped[i]:
        continue
    k = k + 1
    group[k].append(i)
    for j in range(i+1,N_site):
        if have_grouped[j]:
            continue
#        print(i,j)
#        print(cluster_index[j])
#        print(site_index[j])
#        print(coor_data[j])
#        print(coor_data[j].shape)
        if (coor_data[j].shape==coor_data[i].shape):
            if (coor_data[j]==coor_data[i]).all():
                have_grouped[j] = True
                group[k].append(j)
#
N_group = k + 1

In [14]:
fout = open(path_file_work,'w')
print(f"** {dir_src}")
print(f"** {dir_src}", file=fout)
print(f"** There are {N_group} groups. Total sites {N_site}.")
print(f"** There are {N_group} groups. Total sites {N_site}.", file=fout)
for i in range(N_group):
    print()
    print(file=fout)
    print(f"**  Group {i+1} : {group[i]}")
    print(f"**  Group {i+1} : {group[i]}", file=fout)
    print(f"**  Number : {len(group[i])}")
    print(f"**  Number : {len(group[i])}", file=fout)
    print('** coor:', file=fout)
    print(f"** coor :")
    print(f"** coor :", file=fout)
    print(f"{coor_data[group[i][0]]}")
    print(f"{coor_data[group[i][0]]}", file=fout)
    for j in range(len(group[i])):
        print(f"{cluster_index[group[i][j]]} : {site_index[group[i][j]]}")
        print(f"{cluster_index[group[i][j]]} : {site_index[group[i][j]]}", file=fout)
fout.close()

** cu_crystal_different_xyz_WT_untar
** There are 84 groups. Total sites 1902.

**  Group 1 : [0, 111, 222, 333, 444, 548, 559, 570, 665, 678, 682, 711, 715, 722, 724, 727, 730]
**  Number : 17
** coor :
[[1.    3.    2.553 0.   ]
 [2.    3.    3.61  0.   ]
 [3.    3.    4.421 0.   ]
 [4.    3.    5.105 0.   ]]
Cu-bulk : 1
Cu-bulk : 2
Cu-bulk : 3
Cu-bulk : 4
Cu-bulk : 5
Cu-bulk : 6
Cu-bulk : 7
Cu-bulk : 8
Cu-fcc-23 : 1
Cu-fcc-23 : 21
Cu-fcc-23 : 25
Cu-fcc-23 : 51
Cu-fcc-23 : 55
Cu-fcc-23 : 61
Cu-fcc-23 : 63
Cu-fcc-23 : 9
Cu-fcc-23-1 : 11

**  Group 2 : [1, 12, 23, 34, 36, 37, 45, 58, 59, 80, 81, 345, 356, 411, 422, 456, 467, 500, 511, 552, 553, 556, 557, 581, 666, 668, 670, 672, 676, 677, 679, 681, 685, 686, 687, 690, 693, 697, 698, 704, 706, 708, 712, 714, 717, 719, 720, 725]
**  Number : 48
** coor :
[[1.    8.    2.553 0.   ]
 [2.    3.    3.61  0.   ]
 [3.    8.    4.421 0.   ]
 [4.    3.    5.105 0.   ]]
Cu-bulk : 10
Cu-bulk : 11
Cu-bulk : 12
Cu-bulk : 13
Cu-bulk : 131
Cu-bulk : 1

Cu-wulff-size201 : 20
Cu-wulff-size201 : 25
Cu-wulff-size201 : 83
Cu-wulff-size201 : 93

**  Group 53 : [1086, 1107, 1117, 1126, 1134, 1147, 1495, 1513, 1538, 1563, 1598, 1687]
**  Number : 12
** coor :
[[ 1.    12.     2.553  0.   ]
 [ 2.     5.     3.61   0.   ]
 [ 3.    12.     4.421  0.   ]
 [ 4.     4.     5.105  0.   ]]
Cu-fcc-342 : 19
Cu-fcc-342 : 38
Cu-fcc-342 : 47
Cu-fcc-342 : 55
Cu-fcc-342 : 62
Cu-fcc-342 : 74
Cu-fcc-53 : 128
Cu-fcc-53 : 144
Cu-fcc-53 : 167
Cu-fcc-53 : 19
Cu-fcc-53 : 220
Cu-fcc-53 : 99

**  Group 54 : [1174, 1180, 1185, 1191, 1197, 1198, 1200, 1201, 1202, 1207, 1213, 1216, 1217, 1218, 1228, 1229, 1230, 1234, 1237, 1239, 1246, 1249, 1253, 1254]
**  Number : 24
** coor :
[[1.    7.    2.553 0.   ]
 [2.    2.    3.61  0.   ]
 [3.    8.    4.421 0.   ]
 [4.    4.    5.105 0.   ]]
Cu-fcc-42 : 1
Cu-fcc-42 : 15
Cu-fcc-42 : 2
Cu-fcc-42 : 25
Cu-fcc-42 : 30
Cu-fcc-42 : 31
Cu-fcc-42 : 33
Cu-fcc-42 : 34
Cu-fcc-42 : 35
Cu-fcc-42 : 4
Cu-fcc-42 : 45
Cu-fcc-42 : 48
Cu-fcc-42

## 将符合条件的体系（文件夹及其内容）复制到“_symmetry”

In [15]:
#  设定每组提多少个
N_pick_per_group = 1

In [16]:
#  统计有多少个位点被选中
N_pick =0
#
# 对于每一个对称组选取一个体系（组的第一个）即可
for i in range(N_group):
# 选取每个组的前N_pick_per_group个site的index。
    for k in range(N_pick_per_group):
        if k > (len(group[i]) - 1):
            print()
            print('** Group',i,': Only',group[i],'are picked out.')
            k = k -1
            break
#
        j = group[i][k]
        cluster = cluster_index[j]
        site = site_index[j]
#
#  源和目标cluster文件夹
        path_cluster_src = os.path.join(path_src, cluster)
        path_cluster_work = os.path.join(path_work, cluster)
        os.makedirs(path_cluster_work, exist_ok=True)
#
#  源site文件夹
        path_site_src = os.path.join(path_cluster_src, site)
        path_site_work = os.path.join(path_cluster_work, site)
#  检查site目标文件夹是否存在，存在则删除，不然无法copy
        exist = os.path.exists(path_site_work)
        if exist:
            shutil.rmtree(path_site_work)
#  将源site文件夹copy到目标文件夹下
        shutil.copytree(path_site_src, path_site_work)
    N_pick = N_pick + k + 1
#
print()
print('**',N_group,'groups;', N_pick, 'sites are picked out.')


** 84 groups; 84 sites are picked out.
